In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES']='0,3'

In [3]:
import multiprocessing
import pickle
import sys
import argparse
import os
import time

import numpy as np
from numpy import savetxt

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
# from tqdm import tqdm
from tqdm.notebook import tqdm


from src.dataset import CLEVR, collate_data, transform, GQA
from src.model import MACNetwork
# from model_gqa import MACNetwork

In [4]:
batch_size = 128
n_epoch = 25
learning_rate = 1e-4
dim_dict = {'CLEVR': 512,
            'gqa': 2048}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
print( torch.cuda.device_count() )
for i in range(torch.cuda.device_count()):
    print( torch.cuda.get_device_name(i) )

1
Tesla K40m


In [6]:
def accumulate(model1, model2, decay=0.999):
    par1 = dict(model1.named_parameters())
    par2 = dict(model2.named_parameters())

    for k in par1.keys():
        par1[k].data.mul_(decay).add_(1 - decay, par2[k].data)


In [7]:
def train(epoch, dataset_type, dataset_object):
    
#     # loading dataset using hdf5 imposing minimal overhead
#     if dataset_type == "CLEVR":
#         dataset_object = CLEVR('data/CLEVR_v1.0', transform=transform)
#     else:
#         dataset_object = GQA('data/gqa', transform=transform)

    train_set = DataLoader(
        dataset_object, batch_size=batch_size, num_workers=1, collate_fn=collate_data
    )

    dataset = iter(train_set)
    pbar = tqdm(dataset)
    moving_loss = 0

    net.train(True)
    for iter_id, (image, question, q_len, answer) in enumerate(pbar):
        image, question, answer = (
            image.to(device),
            question.to(device),
            answer.to(device),
        )

        net.zero_grad()
        output = net(image, question, q_len)
        loss = criterion(output, answer)
        loss.backward()
        optimizer.step()
        correct = output.detach().argmax(1) == answer
        correct = torch.tensor(correct, dtype=torch.float32).sum() / batch_size

        if moving_loss == 0:
            moving_loss = correct
        else:
            moving_loss = (moving_loss * iter_id + correct) / (iter_id + 1)
#             moving_loss = moving_loss * 0.99 + correct * 0.01 ??? swd

        pbar.set_description('Epoch: {}; CurLoss: {:.5f}; CurAcc: {:.5f}; Tot_Acc: {:.5f}'.format(epoch + 1, loss.item(), correct, moving_loss))

        accumulate(net_running, net, decay)

#     dataset_object.close()


def valid(epoch, dataset_type, dataset_object):
    
#     # loading dataset using hdf5 imposing minimal overhead
#     if dataset_type == "CLEVR":
#         dataset_object = CLEVR('data/CLEVR_v1.0', 'val', transform=None)
#     else:
#         dataset_object = GQA('data/gqa', 'val', transform=None)

    valid_set = DataLoader(
        dataset_object, batch_size=4*batch_size, num_workers=1, collate_fn=collate_data
    )
    dataset = iter(valid_set)

    net_running.train(False)
    correct_counts = 0
    total_counts = 0
    running_loss = 0.0
    batches_done = 0
    with torch.no_grad():
        pbar = tqdm(dataset)
        for image, question, q_len, answer in pbar:
            image, question, answer = (
                image.to(device),
                question.to(device),
                answer.to(device),
            )

            output = net_running(image, question, q_len)
            loss = criterion(output, answer)
            correct = output.detach().argmax(1) == answer
            running_loss += loss.item()

            batches_done += 1
            for c in correct:
                if c:
                    correct_counts += 1
                total_counts += 1

            pbar.set_description('Epoch: {}; Loss: {:.5f}; Acc: {:.5f}'.format(epoch + 1, loss.item(), correct_counts / total_counts))

#     with open('log/log_{}.txt'.format(str(epoch + 1).zfill(2)), 'w') as w:
#         w.write('{:.5f}\n'.format(correct_counts / total_counts))
## NO need to write the log here...
    
    val_acc = correct_counts / total_counts
    val_loss = running_loss / total_counts
    print('Validation Accuracy: {:.5f}'.format(val_acc))
    print('Validation Loss: {:.8f}'.format(val_loss))
    
#     dataset_object.close()
    return val_acc, val_loss

In [8]:
dataset_type = 'CLEVR'
# input
# args = parse_args()
decay = 0.999
load_embd = False
out_name = 'try'

out_directory = 'result/'+ out_name +'/'
if not os.path.exists(out_directory):
    os.makedirs(out_directory)
print('Saving result to: ', out_directory)

Saving result to:  result/try/


In [9]:
if not load_embd:
    with open(f'data/{dataset_type}_dic.pkl', 'rb') as f:
        dic = pickle.load(f)
    n_words = len(dic['word_dic']) + 1
    n_answers = len(dic['answer_dic'])
    print('Training word embeddings from scratch...')
else:
    # add codes for loading GLOVE, embd dimensions, and out dim
    print('Loading GLOVE word embeddings...')
    pass

Training word embeddings from scratch...


In [10]:
# loading dataset using hdf5 imposing minimal overhead
since = time.time()
if dataset_type == "CLEVR":
    train_object = CLEVR('data/CLEVR_v1.0', transform=transform)
    val_object = CLEVR('data/CLEVR_v1.0', 'val', transform=None)
else:
    train_object = GQA('data/gqa', transform=transform)
    val_object = GQA('data/gqa', 'val', transform=None)
print('Dataset loaded in %.2f seconds' %(time.time()-since) )

Dataset loaded in 2.00 seconds


In [11]:
net = MACNetwork(n_words, dim_dict[dataset_type], classes=n_answers, max_step=4)
# net = nn.DataParallel(net)
net.to(device)

net_running = MACNetwork(n_words, dim_dict[dataset_type], classes=n_answers, max_step=4)
# net_running = nn.DataParallel(net_running)
net_running.to(device)

accumulate(net_running, net, 0)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [12]:
# logging training information
with open(out_directory + 'log.txt', 'w') as outfile:
    outfile.write('==== Training details ====\n')
    outfile.write('---- Model structure ----\n')
    outfile.write('Loading GLOVE embedding:  %s.     dictionary dim: %d. \n' %(load_embd, n_words))
    outfile.write('Hidden dimension: %d.     Output dimension: %d.\n' %(dim_dict[dataset_type], n_answers))

    outfile.write('\n---- Training detials ----\n')
    outfile.write('Batch size:  %d.     RA_decay: %f\n' %(batch_size, decay))
    outfile.write('Learning rate: %f.     Epochs: %d\n' %(learning_rate, n_epoch))

In [14]:
learning_curve = np.zeros([0,4])
acc_best = 0

for epoch in range(n_epoch):
    train(epoch, dataset_type, val_object)
    train_acc, train_loss = valid(epoch, dataset_type, val_object)
    val_acc, val_loss = valid(epoch, dataset_type, val_object)

    # saving training result details.
    learning_curve = np.append(learning_curve, np.array([[train_acc,val_acc,train_loss,val_loss]]), axis = 0)
    savetxt(out_directory+'learn_curve.csv', learning_curve, delimiter=',')

    # saving trained models
    if val_acc > acc_best:
        with open(out_directory+'checkpoint.model', 'wb') as f:
    #         with open('checkpoint/checkpoint_{}.model'.format(str(epoch + 1).zfill(2)), 'wb') as f:
            torch.save(net_running.state_dict(), f)
        print('Accuracy increased to %.4f from %.4f, saved to %s. '%(val_acc, acc_best, out_directory+'checkpoint.model'))
        acc_best = val_acc

print('The best validation accuracy: ', acc_best)

/u/tianwei/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Validation Accuracy: 0.42486
Validation Loss: 0.00296304



Validation Accuracy: 0.42486
Validation Loss: 0.00296304
Accuracy increased from 0.4249 to 0.0000, saved to result/try/checkpoint.model. 



Validation Accuracy: 0.50114
Validation Loss: 0.00188413



Validation Accuracy: 0.50114
Validation Loss: 0.00188413
Accuracy increased from 0.5011 to 0.4249, saved to result/try/checkpoint.model. 


KeyboardInterrupt: 

# Training time control:

### using a single K80 GPU, around 1h10min?

### using two K80 GPU:  batch size problem with LSTM.... some internal problem with torch

### using a single K40 GPU:

In [ ]:
## train for val set: 14-15min.
## val for val set: 7-8min